In [2]:
import os
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
import sqlalchemy as sq
import re
from tqdm import tqdm
import jieba

In [3]:
defaultList = []
cedictList = []

In [4]:
# defaultdict = r"""C:\Users\jw\Anaconda3\Lib\site-packages\jieba\dict.txt.big.txt"""
defaultdict = r"""C:\Users\jjw4418\AppData\Local\Continuum\anaconda3\Lib\site-packages\jieba\dict.txt"""

In [5]:
defaultdict

'C:\\Users\\jjw4418\\AppData\\Local\\Continuum\\anaconda3\\Lib\\site-packages\\jieba\\dict.txt'

In [6]:
jieba.set_dictionary(defaultdict)
jieba.load_userdict("cedict.txt")

Building prefix dict from C:\Users\jjw4418\AppData\Local\Continuum\anaconda3\Lib\site-packages\jieba\dict.txt ...
Dumping model to file cache C:\Users\jjw4418\AppData\Local\Temp\jieba.ucc289b289abee56ea5308f1a6d99cc4e.cache
Loading model cost 1.030 seconds.
Prefix dict has been built succesfully.


In [7]:
jieba.set_dictionary(defaultdict)
jieba.load_userdict("cedict.txt")

Building prefix dict from C:\Users\jjw4418\AppData\Local\Continuum\anaconda3\Lib\site-packages\jieba\dict.txt ...
Loading model from cache C:\Users\jjw4418\AppData\Local\Temp\jieba.ucc289b289abee56ea5308f1a6d99cc4e.cache
Loading model cost 0.751 seconds.
Prefix dict has been built succesfully.


In [8]:
data = pd.read_json("core_documents.json")

In [9]:
data.head()

,doc_country,doc_page_name,doc_url,doc_date,doc_content
0,尼日利亚,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,1393891200000,2014年3月3日，驻拉各斯总领事刘侃拜会拉各斯州州长法萧拉。\n\n 刘总领事感谢法萧拉...
1,瑞士,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,1305590400000,2011年5月16日，驻瑞士大使吴恳在大使馆会见到馆的中国侨联代表团和参加座谈的旅瑞华侨华人...
10,阿联酋,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,1360713600000,2013年2月7日，驻阿联酋大使黄杰民出席由阿联酋扎耶德大学孔子学院和英国布莱顿学校阿布扎比...
100,乌克兰,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,1325635200000,2012年1月4日，中乌两国迎来建交20周年纪念日。当天，驻乌克兰大使张喜云在使馆举办中乌建...
1000,奥地利,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,1307664000000,2011年6月8日，驻奥地利大使史明德出席在维也纳大学举行的“2011中国教育展”开幕式。中...


In [10]:
data['doc_date'] = pd.to_datetime(data["doc_date"], unit='ms', origin='unix')

In [11]:
# data.reset_index(level=0, inplace=True)

In [12]:
data.head()

,doc_country,doc_page_name,doc_url,doc_date,doc_content
0,尼日利亚,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,2014-03-04,2014年3月3日，驻拉各斯总领事刘侃拜会拉各斯州州长法萧拉。\n\n 刘总领事感谢法萧拉...
1,瑞士,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,2011-05-17,2011年5月16日，驻瑞士大使吴恳在大使馆会见到馆的中国侨联代表团和参加座谈的旅瑞华侨华人...
10,阿联酋,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,2013-02-13,2013年2月7日，驻阿联酋大使黄杰民出席由阿联酋扎耶德大学孔子学院和英国布莱顿学校阿布扎比...
100,乌克兰,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,2012-01-04,2012年1月4日，中乌两国迎来建交20周年纪念日。当天，驻乌克兰大使张喜云在使馆举办中乌建...
1000,奥地利,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,2011-06-10,2011年6月8日，驻奥地利大使史明德出席在维也纳大学举行的“2011中国教育展”开幕式。中...


In [13]:
data['doc_date'] = data['doc_date'].astype('str').copy()

In [14]:
data.head()

,doc_country,doc_page_name,doc_url,doc_date,doc_content
0,尼日利亚,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,2014-03-04,2014年3月3日，驻拉各斯总领事刘侃拜会拉各斯州州长法萧拉。\n\n 刘总领事感谢法萧拉...
1,瑞士,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,2011-05-17,2011年5月16日，驻瑞士大使吴恳在大使馆会见到馆的中国侨联代表团和参加座谈的旅瑞华侨华人...
10,阿联酋,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,2013-02-13,2013年2月7日，驻阿联酋大使黄杰民出席由阿联酋扎耶德大学孔子学院和英国布莱顿学校阿布扎比...
100,乌克兰,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,2012-01-04,2012年1月4日，中乌两国迎来建交20周年纪念日。当天，驻乌克兰大使张喜云在使馆举办中乌建...
1000,奥地利,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,2011-06-10,2011年6月8日，驻奥地利大使史明德出席在维也纳大学举行的“2011中国教育展”开幕式。中...


In [33]:
countries = [
    '阿尔巴尼亚',
'北马其顿',
'俄罗斯',
'克罗地亚',
'摩尔多瓦',
'塞浦路斯',
'匈牙利',
'爱尔兰',
'比利时',
'法国',
'拉脱维亚',
'摩纳哥',
'圣马力诺',
'意大利',
'爱沙尼亚',
'冰岛',
'梵蒂冈',
'立陶宛',
'挪威',
'斯洛伐克',
'英国',
'安道尔',
'波黑',
'芬兰',
'列支敦士登',
'葡萄牙',
'斯洛文尼亚',
'奥地利',
'波兰',
'荷兰',
'卢森堡',
'瑞典',
'乌克兰',
'保加利亚',
'德国',
'捷克',
'马耳他',
'塞尔维亚',
'希腊',
'白俄罗斯',
'丹麦',
'黑山',
'罗马尼亚',
'瑞士',
'西班牙',
'欧盟',

]

In [37]:
data.loc[data['doc_country'].isin(countries),:]

,doc_country,doc_page_name,doc_url,doc_date,doc_content
1,瑞士,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,2011-05-17,2011年5月16日，驻瑞士大使吴恳在大使馆会见到馆的中国侨联代表团和参加座谈的旅瑞华侨华人...
100,乌克兰,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,2012-01-04,2012年1月4日，中乌两国迎来建交20周年纪念日。当天，驻乌克兰大使张喜云在使馆举办中乌建...
1000,奥地利,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,2011-06-10,2011年6月8日，驻奥地利大使史明德出席在维也纳大学举行的“2011中国教育展”开幕式。中...
10000,希腊,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,2013-07-24,2013年7月23日晚，中国驻希腊大使馆武官韩啸大校在使馆举行“八一”建军节招待会，热烈庆祝...
10001,罗马尼亚,相关新闻,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,2018-12-18,2018年12月18日，外交部副部长、中国－中东欧国家合作（“16+1合作”）秘书处秘书长王...
10002,俄罗斯,相关新闻,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,2014-09-02,当地时间9月1日，国务院副总理张高丽在雅库茨克会见俄罗斯总统普京，并与普京共同出席中俄东线天...
10007,俄罗斯,相关新闻,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,2014-09-01,2014年8月30日，国务院副总理张高丽在莫斯科分别会见了俄罗斯总统能源发展战略和生态安全委...
10008,俄罗斯,相关新闻,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,2014-08-30,当地时间8月30日，中俄能源合作委员会第十一次会议在莫斯科举行。国务院副总理张高丽和俄罗斯副...
10010,俄罗斯,相关新闻,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,2014-07-31,2014年7月31日，外交部长王毅在塔吉克斯坦首都杜尚别会见俄罗斯外长拉夫罗夫。\n 两国...
10011,俄罗斯,相关新闻,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,2014-07-15,当地时间7月14日，国家主席习近平在巴西福塔莱萨会见俄罗斯总统普京。\n 习近平表示，你两...


In [105]:
data.head()

,doc_country,doc_page_name,doc_url,doc_date,doc_content
0,尼日利亚,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,2014-03-04,2014年3月3日，驻拉各斯总领事刘侃拜会拉各斯州州长法萧拉。\n\n 刘总领事感谢法萧拉...
1,瑞士,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,2011-05-17,2011年5月16日，驻瑞士大使吴恳在大使馆会见到馆的中国侨联代表团和参加座谈的旅瑞华侨华人...
10,阿联酋,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,2013-02-13,2013年2月7日，驻阿联酋大使黄杰民出席由阿联酋扎耶德大学孔子学院和英国布莱顿学校阿布扎比...
100,乌克兰,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,2012-01-04,2012年1月4日，中乌两国迎来建交20周年纪念日。当天，驻乌克兰大使张喜云在使馆举办中乌建...
1000,奥地利,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...,2011-06-10,2011年6月8日，驻奥地利大使史明德出席在维也纳大学举行的“2011中国教育展”开幕式。中...


In [106]:
data.shape

(132035, 5)

In [117]:
pcdata = data.loc[data['doc_country'].isin(countries),:].copy()

In [125]:
pcdata['doc_date'].max()

'2018-12-31'

In [126]:
pcdata['doc_date'].min()

'2000-11-07'

In [120]:
pcdata.shape

(30757, 5)

In [121]:
pcdata = pcdata.loc[pcdata['doc_date'] > "1999-12-31",:]

In [122]:
pcdata.shape

(30754, 5)

In [123]:
pcdata = pcdata.loc[pcdata['doc_date'] <= "2018-12-31",:]

In [127]:
pcdata.shape

(30412, 5)

In [128]:
pcdata = pcdata[['doc_country','doc_page_name','doc_url']]

In [129]:
pcdata.head()

,doc_country,doc_page_name,doc_url
1,瑞士,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...
100,乌克兰,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...
1000,奥地利,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...
10000,希腊,驻外报道,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...
10001,罗马尼亚,相关新闻,https://www.fmprc.gov.cn/web/gjhdq_676201/gj_6...


In [130]:
pcdf = pcdata.groupby(['doc_country','doc_page_name']).count()

In [131]:
countrypages = pcdf.unstack("doc_page_name").fillna(0).to_excel("countrypages.xlsx", encoding="utf-8")

In [132]:
countrypages

In [133]:
pages = pd.DataFrame(pcdata.loc[pcdata['doc_country'].isin(countries),:].groupby(["doc_page_name"]).count())
pages.to_excel("pages.xlsx", encoding="utf-8")

In [20]:
data.dtypes

doc_country      object
doc_page_name    object
doc_url          object
doc_date         object
doc_content      object
dtype: object

In [21]:
data.to_json("coreDocumentsLines.json", orient="records", lines=True)

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
data.to_csv("coreDocuments2.csv", encoding='utf-8')

In [ ]:
data.head()

In [ ]:
data.to_csv("coreDocuments2.csv", index_label="index"encoding='utf-8')

In [ ]:
data['doc_content'] = data['doc_content'].astype("str").copy()

In [ ]:
data.dtypes

In [ ]:
data.head()

In [ ]:
data.to_csv("coreDocuments.txt", index=True, encoding='utf-8')

In [ ]:
# data.to_excel("coreDocuments.xlsx", index=True)

In [ ]:
data['doc_content'] = data['doc_content'].str.replace(pat="\W", repl=" ").copy()

In [ ]:
data.head()

In [ ]:
def my_tokenizer(s):
    return jieba.lcut(s, cut_all=False)

count = CountVectorizer(tokenizer=my_tokenizer, min_df=2)

In [ ]:
countdata = count.fit_transform(data['doc_content'])

In [ ]:
countdata

In [ ]:
countDF = pd.DataFrame(countdata.todense(), index=data.index, columns=count.get_feature_names)

In [ ]:
s = data.loc[4,'doc_content']

In [ ]:
seg_list = jieba.cut(s, cut_all=True)
print("Full Mode: " + " ".join(seg_list))  # 全模式
print()
seg_list = jieba.cut(s, cut_all=False)
print("Default Mode: " + " ".join(seg_list))  # 默认模式

In [ ]:
jieba.lcut(s, cut_all=True)

In [ ]:
len(count.get_feature_names())